In [1]:
"""
Synthetic Airline Churn Dataset Generator (50,000 rows)
Meets the UPDATED PROMPT requirements: overlap, noise, moderate correlations, archetypes, validation.

Saves: /mnt/data/airline_synthetic_50k_updated.csv
"""

import numpy as np
import pandas as pd
from scipy.stats import norm, beta, expon, poisson
from scipy.stats import lognorm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
import math
import random
import warnings

warnings.filterwarnings("ignore")

RND = 87
np.random.seed(RND)
random.seed(RND)

N = 100
BASE_CHURN_RATE = 0.40

# ---------- Helper functions ----------
def make_corr_matrix(dim, low=0.25, high=0.45, rnd=RND):
    """
    Make a symmetric positive-definite correlation matrix with values in [low, high] for off-diagonals.
    We create a random matrix and blend towards identity to control eigenvalues.
    """
    rng = np.random.default_rng(rnd)
    A = rng.normal(size=(dim, dim))
    mat = np.dot(A, A.T)
    # scale to correlation
    D = np.sqrt(np.diag(mat))
    corr = mat / np.outer(D, D)
    # move off-diagonals towards random values in [low,high]
    off = rng.uniform(low, high, size=(dim, dim))
    off = np.triu(off, 1)
    off = off + off.T
    np.fill_diagonal(off, 1.0)
    # blend using alpha to keep PD
    alpha = 0.6
    blended = alpha * corr + (1 - alpha) * off
    # ensure PD by adding small jitter
    eigmin = np.linalg.eigvalsh(blended).min()
    if eigmin <= 0:
        blended += np.eye(dim) * (abs(eigmin) + 1e-6)
    return blended

def transform_normal_to_beta(z, a, b):
    """Map standard normal variable -> uniform via CDF -> beta via ppf."""
    u = norm.cdf(z)
    return beta.ppf(u, a, b)

# zero-inflated poisson sampling helper
def zero_inflated_poisson(size, p_zero, lam):
    res = np.zeros(size, dtype=int)
    u = np.random.rand(size)
    nonzero_idx = u >= p_zero
    res[nonzero_idx] = poisson.rvs(mu=lam, size=nonzero_idx.sum())
    return res

# clamp helper
def clamp(arr, low, high):
    arr = np.array(arr)
    arr = np.where(np.isnan(arr), np.nan, arr)
    arr = np.minimum(np.maximum(arr, low), high)
    return arr

# ---------- Feature list in specified order ----------
features = [
    "booking_freq_change",         # continuous -1.0..1.0
    "fare_downgrade_trend",       # beta 0..1
    "booking_abandon_rate",       # 0..0.95
    "recent_disruption_count",    # integer 0..8 (zero-inflated)
    "recent_service_issues",      # integer 0..6 (zero-inflated)
    "lounge_access_issues",       # integer 0..4 (zero-inflated, ~20% customers have nonzero potential)
    "app_usage_drop",             # -0.3..1.0
    "email_engagement_drop",      # 0..1.0
    "loyalty_inactivity_days",    # 0..365
    "ancillary_spend_drop",       # -0.4..1.0
    "upgrade_acceptance_drop",    # 0..1.0 (30% frequent upgraders)
    "spend_per_trip_decline",     # -0.4..1.0
    "frustration_score_llm",      # 0..1.0
    "complaint_frequency_change", # -0.5..4.0
    "churn_intent_signal_llm"     # 0..1.0
]

D = len(features)

# ---------- Step 1: Create churn labels with base rate and archetype assignment ----------
# We'll sample churn label first with base rate, but we will later produce features conditional on label
churn = (np.random.rand(N) < BASE_CHURN_RATE).astype(int)

# Keep indices for churners/retained
idx_churn = np.where(churn == 1)[0]
idx_retained = np.where(churn == 0)[0]
n_churn = len(idx_churn)
n_retained = len(idx_retained)
print(f"Initial churn count: {n_churn} ({n_churn/N:.3%})")

# ---------- Step 2: Create conditional latent Gaussian for features to induce correlations ----------
# We'll create two latent MVNs (retained/churn) with slightly different means to reflect per-spec means,
# but still overlapping heavily.

# desired mean shifts for churn vs retained for a subset of features (based on prompt)
# We'll specify mean vectors (in latent space) for each label to roughly cause the marginals differences.
# Values are approximate and small to keep overlap.

latent_mean_retained = np.zeros(D)
latent_mean_churn = np.zeros(D)

# Map per-feature desired label-level trends (these are *latent mean shifts*, not final feature values)
# Positive latent shift will tend to produce higher feature values after transformation (we'll handle mapping specifics).
# For features where churn mean > retained mean, set churn mean > retained mean.

# booking_freq_change: churn mean lower -> negative shift
latent_mean_retained[0] = 0.06
latent_mean_churn[0] = -0.15

# fare_downgrade_trend: churn higher
latent_mean_retained[1] = -0.2
latent_mean_churn[1] = 0.15

# booking_abandon_rate: churn higher
latent_mean_retained[2] = -0.15
latent_mean_churn[2] = 0.15

# recent_disruption_count: churn slightly higher
latent_mean_retained[3] = -0.1
latent_mean_churn[3] = 0.05

# recent_service_issues
latent_mean_retained[4] = -0.12
latent_mean_churn[4] = 0.08

# lounge_access_issues (rare): keep small shifts
latent_mean_retained[5] = -0.05
latent_mean_churn[5] = 0.02

# app_usage_drop
latent_mean_retained[6] = -0.05
latent_mean_churn[6] = 0.25

# email_engagement_drop
latent_mean_retained[7] = -0.05
latent_mean_churn[7] = 0.18

# loyalty_inactivity_days -> larger shift for churn
latent_mean_retained[8] = -0.08
latent_mean_churn[8] = 0.20

# ancillary_spend_drop
latent_mean_retained[9] = -0.05
latent_mean_churn[9] = 0.18

# upgrade_acceptance_drop (30% frequent upgraders will be handled later)
latent_mean_retained[10] = -0.02
latent_mean_churn[10] = 0.12

# spend_per_trip_decline
latent_mean_retained[11] = -0.05
latent_mean_churn[11] = 0.15

# frustration_score_llm
latent_mean_retained[12] = -0.15
latent_mean_churn[12] = 0.20

# complaint_frequency_change
latent_mean_retained[13] = -0.05
latent_mean_churn[13] = 0.12

# churn_intent_signal_llm
latent_mean_retained[14] = -0.20
latent_mean_churn[14] = 0.30

# Correlation/covariance: moderate correlations
corr = make_corr_matrix(D, low=0.25, high=0.45, rnd=RND)
# produce covariance by setting unit variances for latent Gaussian
cov = corr.copy()

# Sample latent vectors conditional on churn label
latent = np.zeros((N, D))
# Retained
latent[idx_retained] = np.random.multivariate_normal(latent_mean_retained, cov, size=n_retained)
# Churners
latent[idx_churn] = np.random.multivariate_normal(latent_mean_churn, cov, size=n_churn)

# For added uncertainty, add small Gaussian noise
latent += np.random.normal(scale=0.02, size=latent.shape)

# ---------- Step 3: Map latent variables to target marginals per feature per label ----------
# We'll convert latent -> uniform via CDF of normal -> to desired distribution via inverse CDF (ppf) where applicable.
u = norm.cdf(latent)  # uniform 0..1 matrix

# Prepare result dataframe
df = pd.DataFrame(index=np.arange(N), columns=["customer_id"] + features + ["churn_label"])
df["customer_id"] = np.arange(1000000, 1000000 + N)
df["churn_label"] = churn

# 1) booking_freq_change (normal-like in [-1,1], churners mean -0.15 std0.30, retained mean 0.05 std0.35)
# We'll use inverse CDF of normal with those params per label
booking = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        booking[i] = norm.ppf(u[i, 0], loc=-0.15, scale=0.30)
    else:
        booking[i] = norm.ppf(u[i, 0], loc=0.05, scale=0.35)
booking = clamp(booking, -1.0, 1.0)
df["booking_freq_change"] = np.round(booking, 4)

# 2) fare_downgrade_trend (beta: churn Beta(3,4), retained Beta(2,5))
fare = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        fare[i] = beta.ppf(u[i, 1], 3, 4)
    else:
        fare[i] = beta.ppf(u[i, 1], 2, 5)
fare = clamp(fare, 0.0, 1.0)
df["fare_downgrade_trend"] = np.round(fare, 4)

# 3) booking_abandon_rate (churn Beta(3,3), retained Beta(2,5)) bounded 0..0.95
abandon = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        abandon[i] = beta.ppf(u[i, 2], 3, 3)
    else:
        abandon[i] = beta.ppf(u[i, 2], 2, 5)
abandon = clamp(abandon, 0.0, 0.95)
df["booking_abandon_rate"] = np.round(abandon, 4)

# 4) recent_disruption_count: zero-inflated Poisson
# We'll use latent u values to determine zero-inflation threshold and quantiles for Poisson lambda
disrupt = np.zeros(N, dtype=int)
for i in range(N):
    if churn[i] == 1:
        p_zero = 0.40
        lam = 1.5
    else:
        p_zero = 0.55
        lam = 0.9
    if np.random.rand() < p_zero:
        disrupt[i] = 0
    else:
        disrupt[i] = int(poisson.ppf(u[i, 3], mu=lam))
disrupt = np.clip(disrupt, 0, 8)
df["recent_disruption_count"] = disrupt

# 5) recent_service_issues: zero-inflated Poisson
svc = np.zeros(N, dtype=int)
for i in range(N):
    if churn[i] == 1:
        p_zero = 0.50
        lam = 1.2
    else:
        p_zero = 0.70
        lam = 0.6
    if np.random.rand() < p_zero:
        svc[i] = 0
    else:
        svc[i] = int(poisson.ppf(u[i, 4], mu=lam))
svc = np.clip(svc, 0, 6)
df["recent_service_issues"] = svc

# 6) lounge_access_issues: only relevant for ~20% of customers (premium-like). We'll pick 20% "eligible".
eligible_lounge = (np.random.rand(N) < 0.20)
lounge = np.zeros(N, dtype=int)
for i in range(N):
    if not eligible_lounge[i]:
        lounge[i] = 0
    else:
        if churn[i] == 1:
            # churners: 75% zero among eligible, else uniform 1-3
            if np.random.rand() < 0.75:
                lounge[i] = 0
            else:
                lounge[i] = np.random.randint(1, 4)
        else:
            if np.random.rand() < 0.85:
                lounge[i] = 0
            else:
                lounge[i] = np.random.randint(1, 3)
lounge = np.clip(lounge, 0, 4)
df["lounge_access_issues"] = lounge

# 7) app_usage_drop: churn centered ~0.40 std0.25, retained ~0.15 std0.28, bound -0.3..1.0
app = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        app[i] = norm.ppf(u[i, 6], loc=0.40, scale=0.25)
    else:
        app[i] = norm.ppf(u[i, 6], loc=0.15, scale=0.28)
app = clamp(app, -0.3, 1.0)
df["app_usage_drop"] = np.round(app, 4)

# 8) email_engagement_drop: churn mean 0.55 std0.25; retained mean 0.35 std0.30
email = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        email[i] = norm.ppf(u[i, 7], loc=0.55, scale=0.25)
    else:
        email[i] = norm.ppf(u[i, 7], loc=0.35, scale=0.30)
email = clamp(email, 0.0, 1.0)
df["email_engagement_drop"] = np.round(email, 4)

# 9) loyalty_inactivity_days: exponential-ish bounded 0..365
loyal = np.zeros(N, dtype=int)
for i in range(N):
    if churn[i] == 1:
        # create right-skewed values with median ~90
        val = expon.ppf(u[i, 8], scale=120)  # scale tuned to produce median ~90
    else:
        val = expon.ppf(u[i, 8], scale=60)   # median ~42
    loyal[i] = int(min(max(0, val), 365))
df["loyalty_inactivity_days"] = loyal

# 10) ancillary_spend_drop: churn mean 0.35 std0.30; retained mean 0.15 std0.28; clamp -0.4..1.0
anc = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        anc[i] = norm.ppf(u[i, 9], loc=0.35, scale=0.30)
    else:
        anc[i] = norm.ppf(u[i, 9], loc=0.15, scale=0.28)
anc = clamp(anc, -0.4, 1.0)
df["ancillary_spend_drop"] = np.round(anc, 4)

# 11) upgrade_acceptance_drop: only meaningful for ~30% frequent upgraders; others uniform 0..0.3
freq_upgraders = (np.random.rand(N) < 0.30)
upgrade = np.zeros(N)
for i in range(N):
    if not freq_upgraders[i]:
        upgrade[i] = np.random.uniform(0.0, 0.3)
    else:
        if churn[i] == 1:
            upgrade[i] = np.random.normal(0.55, 0.25)
        else:
            upgrade[i] = np.random.normal(0.30, 0.30)
upgrade = clamp(upgrade, 0.0, 1.0)
df["upgrade_acceptance_drop"] = np.round(upgrade, 4)

# 12) spend_per_trip_decline: churn mean 0.30 std0.32; retained mean 0.10 std0.30; clamp -0.4..1.0
spend = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        spend[i] = norm.ppf(u[i, 11], loc=0.30, scale=0.32)
    else:
        spend[i] = norm.ppf(u[i, 11], loc=0.10, scale=0.30)
# correlate with ancillary_spend_drop mildly (done post)
spend = clamp(spend, -0.4, 1.0)
df["spend_per_trip_decline"] = np.round(spend, 4)

# 13) frustration_score_llm: churn Beta(4,4) mean 0.5; retained Beta(2,6) mean 0.25
frust = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        frust[i] = beta.ppf(u[i, 12], 4, 4)
    else:
        frust[i] = beta.ppf(u[i, 12], 2, 6)
frust = clamp(frust, 0.0, 1.0)
df["frustration_score_llm"] = np.round(frust, 4)

# 14) complaint_frequency_change: log-normal-like (allow negative small values but mostly positive)
compl = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        # lognormal with mean around 0.8, std wide
        s = 0.9
        scale = math.exp(0.6)  # shift for mean
        compl[i] = lognorm.ppf(u[i, 13], s=s, scale=scale)
    else:
        s = 0.7
        scale = math.exp(0.25)
        compl[i] = lognorm.ppf(u[i, 13], s=s, scale=scale)
# occasionally allow slight negative small values to match -0.5 lower bound (clip)
compl = clamp(compl, -0.5, 4.0)
df["complaint_frequency_change"] = np.round(compl, 4)

# 15) churn_intent_signal_llm: churn Beta(5,3) mean ~0.62, retained Beta(2,8) mean ~0.20
intent = np.zeros(N)
for i in range(N):
    if churn[i] == 1:
        intent[i] = beta.ppf(u[i, 14], 5, 3)
    else:
        intent[i] = beta.ppf(u[i, 14], 2, 8)
intent = clamp(intent, 0.0, 1.0)
df["churn_intent_signal_llm"] = np.round(intent, 4)

# ---------- Step 4: Inject the moderate cross-feature correlations specified ----------
# We already used a latent MVN to impose moderate correlations. Now nudge spending cluster correlations a bit stronger:
# spending cluster features indices: fare_downgrade_trend(1), ancillary_spend_drop(9), spend_per_trip_decline(11)
# We'll make spend_per_trip_decline slightly correlated with anc & fare:
df["spend_per_trip_decline"] = np.round(
    0.52 * df["ancillary_spend_drop"].fillna(0) +
    0.28 * df["fare_downgrade_trend"].fillna(0) +
    0.2 * df["spend_per_trip_decline"], 4
)
df["spend_per_trip_decline"] = clamp(df["spend_per_trip_decline"], -0.4, 1.0)

# Engagement cluster: app_usage_drop, email_engagement_drop, loyalty_inactivity_days (scale loyalty to 0..1 for blending)
loyal_scaled = df["loyalty_inactivity_days"] / 365.0
df["email_engagement_drop"] = np.round(
    0.55 * df["app_usage_drop"] + 0.20 * loyal_scaled + 0.25 * df["email_engagement_drop"], 4
)
df["email_engagement_drop"] = clamp(df["email_engagement_drop"], 0.0, 1.0)

# frustration <-> service_issues mild link:
df["frustration_score_llm"] = np.round(
    0.55 * df["frustration_score_llm"] + 0.12 * (df["recent_service_issues"]/ (1 + df["recent_service_issues"])), 4
)
df["frustration_score_llm"] = clamp(df["frustration_score_llm"], 0.0, 1.0)

# churn_intent_signal_llm slightly correlated with frustration
df["churn_intent_signal_llm"] = np.round(
    0.68 * df["churn_intent_signal_llm"] + 0.18 * df["frustration_score_llm"], 4
)
df["churn_intent_signal_llm"] = clamp(df["churn_intent_signal_llm"], 0.0, 1.0)

# ---------- Step 5: Create archetypes among churners and retained with mixed signals ----------
# We will alter features for slice of churners/retained to create your named archetypes.
arche_churners = {
    "Silent Leavers": 0.25,  # 25% of churners
    "Price Switchers": 0.30,
    "Service-Driven": 0.20,
    "Disengaged Drifters": 0.25
}
arche_retained = {
    "Loyal Despite Issues": 0.20,  # 20% of retained
    "Value Optimizers": 0.25
}

# Choose churner indices and assign archetypes
remaining = idx_churn.copy().tolist()
np.random.shuffle(remaining)
start = 0
arche_idx_map = {}
for name, frac in arche_churners.items():
    cnt = int(np.round(n_churn * frac))
    sel = remaining[start:start+cnt]
    start += cnt
    arche_idx_map[name] = sel

# Apply archetype-specific adjustments (add noise + shift)
# Silent Leavers: low frustration & complaint & intent, but booking decline
for i in arche_idx_map["Silent Leavers"]:
    df.at[i, "frustration_score_llm"] = np.round(np.random.uniform(0.1, 0.3), 4)
    df.at[i, "complaint_frequency_change"] = np.round(np.random.uniform(0.0, 0.2), 4)
    df.at[i, "churn_intent_signal_llm"] = np.round(np.random.uniform(0.0, 0.3), 4)
    df.at[i, "booking_freq_change"] = np.round(np.random.uniform(-0.55, -0.15), 4)
    # small app drop
    df.at[i, "app_usage_drop"] = np.round(np.random.uniform(0.1, 0.5), 4)

# Price Switchers
for i in arche_idx_map["Price Switchers"]:
    df.at[i, "fare_downgrade_trend"] = np.round(np.random.uniform(0.6, 0.9), 4)
    df.at[i, "booking_abandon_rate"] = np.round(np.random.uniform(0.5, 0.85), 4)
    df.at[i, "frustration_score_llm"] = np.round(np.random.uniform(0.3, 0.6), 4)
    # spending signals
    df.at[i, "ancillary_spend_drop"] = np.round(np.random.uniform(0.25, 0.7), 4)

# Service-Driven Churners
for i in arche_idx_map["Service-Driven"]:
    df.at[i, "recent_disruption_count"] = np.random.randint(3, 7)
    df.at[i, "recent_service_issues"] = np.random.randint(2, 5)
    df.at[i, "frustration_score_llm"] = np.round(np.random.uniform(0.6, 0.95), 4)
    df.at[i, "complaint_frequency_change"] = np.round(np.random.uniform(1.5, 3.0), 4)
    df.at[i, "churn_intent_signal_llm"] = np.round(np.random.uniform(0.3, 0.7), 4)

# Disengaged Drifters
for i in arche_idx_map["Disengaged Drifters"]:
    df.at[i, "loyalty_inactivity_days"] = int(np.random.randint(150, 301))
    df.at[i, "app_usage_drop"] = np.round(np.random.uniform(0.6, 0.95), 4)
    df.at[i, "email_engagement_drop"] = np.round(np.random.uniform(0.6, 0.95), 4)
    # low complaints/tools
    df.at[i, "frustration_score_llm"] = np.round(np.random.uniform(0.2, 0.5), 4)

# Retained archetypes
ret_idx_list = idx_retained.tolist()
np.random.shuffle(ret_idx_list)
start = 0
for name, frac in arche_retained.items():
    cnt = int(np.round(n_retained * frac))
    sel = ret_idx_list[start:start+cnt]
    start += cnt
    if name == "Loyal Despite Issues":
        for i in sel:
            df.at[i, "recent_service_issues"] = np.random.randint(1, 4)
            df.at[i, "frustration_score_llm"] = np.round(np.random.uniform(0.3, 0.5), 4)
            df.at[i, "booking_freq_change"] = np.round(np.random.uniform(-0.1, 0.2), 4)
    elif name == "Value Optimizers":
        for i in sel:
            df.at[i, "fare_downgrade_trend"] = np.round(np.random.uniform(0.4, 0.6), 4)
            df.at[i, "booking_abandon_rate"] = np.round(np.random.uniform(0.4, 0.7), 4)
            # they still stay
            df.at[i, "churn_intent_signal_llm"] = np.round(np.random.uniform(0.0, 0.2), 4)

# ---------- Step 6: Add realistic noise, missing values, outliers ----------
# Add noise to continuous fields to ensure no single feature dominates
for col in ["booking_freq_change","fare_downgrade_trend","booking_abandon_rate",
            "app_usage_drop","email_engagement_drop","ancillary_spend_drop",
            "spend_per_trip_decline","frustration_score_llm","churn_intent_signal_llm"]:
    noise = np.random.normal(0, 0.03, size=N)
    df[col] = np.round(df[col].astype(float) + noise, 4)
    # re-clamp
    if col == "booking_freq_change":
        df[col] = clamp(df[col], -1.0, 1.0)
    elif col in ["fare_downgrade_trend","booking_abandon_rate","email_engagement_drop",
                 "ancillary_spend_drop","spend_per_trip_decline","frustration_score_llm","churn_intent_signal_llm"]:
        df[col] = clamp(df[col], 0.0, 1.0)

# Missing values: 2.5% cells missing across features (not churn_label)
feat_cols = features.copy()
total_cells = N * len(feat_cols)
n_missing = int(total_cells * 0.025)
for _ in range(n_missing):
    i = np.random.randint(0, N)
    c = random.choice(feat_cols)
    df.at[i, c] = np.nan

# Outliers: ~1.5% of rows, apply extreme but realistic values
n_outliers = int(N * 0.015)
outlier_cols = ["recent_disruption_count","recent_service_issues","loyalty_inactivity_days","complaint_frequency_change"]
for _ in range(n_outliers):
    i = np.random.randint(0, N)
    c = random.choice(outlier_cols)
    if c in ["recent_disruption_count"]:
        df.at[i, c] = int(np.random.randint(6, 9))
    elif c == "recent_service_issues":
        df.at[i, c] = int(np.random.randint(5, 7))
    elif c == "loyalty_inactivity_days":
        df.at[i, c] = int(np.random.randint(200, 365))
    else:
        df.at[i, c] = round(min(4.0, np.random.exponential(2.0)), 3)

# Ensure integer columns are ints and bounds respected
for c in ["recent_disruption_count","recent_service_issues","lounge_access_issues","loyalty_inactivity_days"]:
    df[c] = df[c].fillna(0).astype(int)
# ensure churn_label int
df["churn_label"] = df["churn_label"].astype(int)

# ---------- Step 7: Validation checks ----------
print("\nValidation checks:")

# 1) Feature-target correlations (point biserial approximated by Pearson with binary)
corrs = {}
for col in feat_cols:
    try:
        corr = df[[col, "churn_label"]].dropna().corr().iloc[0,1]
    except Exception:
        corr = np.nan
    corrs[col] = corr
sorted_by_abs = sorted(corrs.items(), key=lambda x: abs(x[1]) if not math.isnan(x[1]) else -1, reverse=True)
print("Top feature correlations with churn (abs):")
for k,v in sorted_by_abs[:10]:
    print(f"  {k}: {v:.3f}")

max_abs_corr = max(abs(v) for v in list(corrs.values()) if not math.isnan(v))
print(f"Max absolute feature correlation with churn: {max_abs_corr:.3f} (target <= 0.50)")

# 2) Inter-feature correlation matrix (check multicollinearity)
num_df = df[feat_cols].copy()
num_df = num_df.apply(pd.to_numeric, errors='coerce')
corr_mat = num_df.corr().fillna(0)
max_pair_corr = corr_mat.where(~np.eye(len(corr_mat),dtype=bool)).abs().max().max()
print(f"Max pairwise feature correlation: {max_pair_corr:.3f} (target 0.20-0.55)")

# 3) Train a small logistic model to compute predicted probabilities & feature importances (coefs)
# Drop rows with NaNs for model training (a small fraction)
model_df = df.dropna(subset=feat_cols + ["churn_label"]).copy()
X = model_df[feat_cols].values
y = model_df["churn_label"].values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
# Use LogisticRegression with L2 regularization and calibration to get probabilities
clf = LogisticRegression(max_iter=200, class_weight='balanced', solver='lbfgs', random_state=RND)
cal = CalibratedClassifierCV(clf, method='isotonic', cv=5)
cal.fit(Xs, y)
probs = cal.predict_proba(Xs)[:,1]

# Ambiguity zone: fraction with prob between 0.25 and 0.75
ambig_frac = ((probs >= 0.25) & (probs <= 0.75)).mean()
print(f"Ambiguity zone (pred prob in [0.25,0.75]): {ambig_frac:.3%} (target 30-40%)")

# Feature importance (absolute coef from logistic on scaled features)
base_clf = LogisticRegression(max_iter=200, class_weight='balanced', solver='lbfgs', random_state=RND)
base_clf.fit(Xs, y)
coefs = np.abs(base_clf.coef_[0])
feat_importance = sorted(zip(feat_cols, coefs), key=lambda x: x[1], reverse=True)
print("\nTop 10 feature importances (abs coef):")
for f, c in feat_importance[:10]:
    print(f"  {f}: {c:.4f}")

# Ratio between top and 10th: ensure no 1-2 dominating features
top10_vals = [v for _, v in feat_importance[:10]]
if len(top10_vals) >= 2:
    ratio_top = top10_vals[0] / (top10_vals[-1] + 1e-9)
else:
    ratio_top = float('inf')
print(f"Top-to-10th importance ratio ~ {ratio_top:.2f} (should be within ~2x ideally)")

# 4) Predicted probabilities distribution check (rough uniform-ish not bimodal)
import matplotlib.pyplot as plt
# (we don't display plot here, but compute some metrics)
hist, bins = np.histogram(probs, bins=20, range=(0,1))
hist_uniformity = (np.std(hist) / (np.mean(hist) + 1e-9))
print(f"Predicted-prob histogram std/mean: {hist_uniformity:.3f} (lower means more uniform)")

# 5) Fraction of ambiguous customers in dataset by applying model to all available rows (we used training set)
print("\nSummary stats:")
print(f" - Rows generated: {N}")
print(f" - Churn fraction (requested 18%): {df['churn_label'].mean():.3%}")
print(f" - Rows used in model (no-NaN): {len(model_df)}")
print(f" - Ambiguity fraction (model preds in [0.25,0.75]): {ambig_frac:.3%}")
print(f" - Max feature correlation with churn: {max_abs_corr:.3f}")
print(f" - Max pairwise feature corr: {max_pair_corr:.3f}")

# ---------- Step 8: Final touches and save ----------
out_path = "airline_synthetic_test.csv"
# Reorder columns as requested
cols_out = ["customer_id"] + feat_cols + ["churn_label"]
df_out = df[cols_out].copy()
df_out.to_csv(out_path, index=False)
print(f"\nSaved CSV to: {out_path}")

# Provide minor sanity sample output
print("\nSample rows (first 10):")
print(df_out.head(10).to_string(index=False))


Initial churn count: 43 (43.000%)

Validation checks:
Top feature correlations with churn (abs):
  churn_intent_signal_llm: 0.724
  email_engagement_drop: 0.644
  frustration_score_llm: 0.608
  fare_downgrade_trend: 0.582
  spend_per_trip_decline: 0.580
  ancillary_spend_drop: 0.568
  app_usage_drop: 0.545
  loyalty_inactivity_days: 0.516
  booking_freq_change: -0.484
  booking_abandon_rate: 0.434
Max absolute feature correlation with churn: 0.724 (target <= 0.50)
Max pairwise feature correlation: 0.878 (target 0.20-0.55)
Ambiguity zone (pred prob in [0.25,0.75]): 2.667% (target 30-40%)

Top 10 feature importances (abs coef):
  fare_downgrade_trend: 1.2873
  frustration_score_llm: 1.1349
  churn_intent_signal_llm: 1.1184
  booking_freq_change: 1.0437
  complaint_frequency_change: 1.0132
  email_engagement_drop: 0.6028
  upgrade_acceptance_drop: 0.5332
  spend_per_trip_decline: 0.4121
  app_usage_drop: 0.3795
  booking_abandon_rate: 0.3321
Top-to-10th importance ratio ~ 3.88 (should be 